# Getting Started
#### To start this notebook, you must have a huggingface account and request access from Meta to use Llama 2.
https://huggingface.co/

#### In huggingface, create an access token
https://huggingface.co/docs/hub/security-tokens

#### Inside your home directory access .apikeys and create a huggingface_api_key.txt and paste you access token inside the file
path - /home/{your username}/.apikeys
#### Using the link below request access from Meta
https://huggingface.co/meta-llama/Llama-2-7b-hf

#### Once you recieve access from Meta inside terminal create a conda environment using
conda create --name {environment_name} python=3.10

#### Then Install ipykernel using
conda install ipykernel

#### To allow your environment to be used in the notebook run the following line and select your environment on the top right besides the debugging symbol
python -m ipykernel install --user --name={environment_name}

#### Go back to terminal and install all the packages with
pip install -r packages.txt

#### Edit the data set, test set, and validation set under Load Datasets with the path and you are good to go!
##### All imported data must be a csv. Csv must have at least 2 columns for output and test
##### Import the data inside the llama-recipes folder

#### If you are just loading a model run the first 4 cells then skip to the loading section

## Access Huggingface API Key

In [5]:
# Set cache directory and load Huggingface api key
import os

username = os.getenv('USER')
directory_path = os.path.join('/scratch', username)
output_dir = os.path.join('/scratch', username ,'llama-output')

# Set Huggingface cache directory to be on scratch drive
if os.path.exists(directory_path):
    hf_cache_dir = os.path.join(directory_path, 'hf_cache')
    if not os.path.exists(hf_cache_dir):
        os.mkdir(hf_cache_dir)
    print(f"Okay, using {hf_cache_dir} for huggingface cache. Models will be stored there.")
    assert os.path.exists(hf_cache_dir)
    os.environ['TRANSFORMERS_CACHE'] = f'/scratch/{username}/hf_cache/'
else:
    error_message = f"Are you sure you entered your username correctly? I couldn't find a directory {directory_path}."
    raise FileNotFoundError(error_message)

# Load Huggingface api key
api_key_loc = os.path.join('/home', username, '.apikeys', 'huggingface_api_key.txt')

if os.path.exists(api_key_loc):
    print('Huggingface API key loaded.')
    with open(api_key_loc, 'r') as api_key_file:
        huggingface_api_key = api_key_file.read().strip()  # Read and store the contents
else:
    error_message = f'Huggingface API key not found. You need to get an HF API key from the HF website and store it at {api_key_loc}.\n' \
                    'The API key will let you download models from Huggingface.'
    raise FileNotFoundError(error_message)

Okay, using /scratch/kwamea/hf_cache for huggingface cache. Models will be stored there.
Huggingface API key loaded.


In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline, DataCollatorWithPadding, default_data_collator, Trainer, TrainingArguments
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
from configs import fsdp_config, train_config
from peft import get_peft_model, prepare_model_for_int8_training, PeftModelForCausalLM, LoraConfig, TaskType, prepare_model_for_int8_training, PeftModel
from utils.dataset_utils import get_preprocessed_dataset
from utils.train_utils import (
    train,
    freeze_transformer_layers,
    setup,
    setup_environ_flags,
    clear_gpu_cache,
    print_model_size,
    #get_policies
)
from utils.config_utils import (
    update_config,
    generate_peft_config,
    generate_dataset_config,
)
from datasets import load_dataset
from datasets import Dataset
from pathlib import Path
import sys
import csv
import random
import json
from configs.datasets import samsum_dataset, alpaca_dataset, grammar_dataset
from ft_datasets.utils import Concatenator

## Load Llama 2 7B

In [7]:
tokenizer = AutoTokenizer.from_pretrained(
        "meta-llama/Llama-2-7b-hf",
        cache_dir=os.path.join('/scratch', username),
        load_in_8bit=True if train_config.quantization else None,
        token=huggingface_api_key,
)

tokenizer.add_special_tokens(
    {
        "pad_token": "<PAD>",
    }
)

model = AutoModelForCausalLM.from_pretrained(
        "meta-llama/Llama-2-7b-hf",
        load_in_8bit=True if train_config.quantization else None,
        device_map="auto" if train_config.quantization else None,
        cache_dir=os.path.join('/scratch', username),
        token=huggingface_api_key
)
#the code will output "Error displaying widget: model not found" it is not an error just the code failing to create a loading bar

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Load in your dataset

In [13]:
#add testset and rename current test set to validation set 

#edit file path to your unique dataset
dataset = load_dataset('csv', data_files='samsum-data/samsum-train.csv',split = 'train')
valset = load_dataset('csv', data_files='samsum-data/samsum-validation.csv',split = 'train')
testset = load_dataset('csv', data_files='samsum-data/samsum-test.csv',split = 'train')


#Sampling only for testing
sample_fraction = 0.10

num_samples = int(len(dataset) * sample_fraction)
num_s = int(len(valset) * sample_fraction)
num_sa = int(len(testset) * sample_fraction)

# Sample 1/10 of the data randomly
dataset = dataset.shuffle(seed=42).select(list(range(num_samples)))
valset = valset.shuffle(seed=42).select(list(range(num_s)))
testset = testset.shuffle(seed=42).select(list(range(num_sa)))

#Edit the prompt to tell the model what to do
prompt = (
    f"Summarize this dialog:\n{{dialog}}\n---\nSummary:\n{{summary}}{{eos_token}}"
)

#edit the variables in prompt.format to match your data: essentially what you what the model to read
def apply_prompt_template(sample):
    return {
        "text": prompt.format(
            dialog = sample["dialogue"],
            summary = sample["summary"],
            eos_token=tokenizer.eos_token,
        )
    }

dataset = dataset.map(apply_prompt_template, remove_columns=list(dataset.features))
valset = valset.map(apply_prompt_template, remove_columns=list(valset.features))
testSet = testset.map(apply_prompt_template, remove_columns=list(testset.features))

dataset = dataset.map(
    lambda sample: tokenizer(sample["text"]),
    batched=True,
    remove_columns=list(dataset.features), 
).map(Concatenator(), batched=True)
valset = valset.map(
    lambda sample: tokenizer(sample["text"]),
    batched=True,
    remove_columns=list(valset.features), 
).map(Concatenator(), batched=True)
testset = testset.map(
    lambda sample: tokenizer(sample["text"]),
    batched=True,
    remove_columns=list(testset.features), 
).map(Concatenator(), batched=True)


train_dataset = dataset
val_dataset = valset
test_dataset = testSet

Found cached dataset csv (/home/kwamea/.cache/huggingface/datasets/csv/default-04ed86c0369d6106/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Found cached dataset csv (/home/kwamea/.cache/huggingface/datasets/csv/default-98e7740e620c6c6f/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Found cached dataset csv (/home/kwamea/.cache/huggingface/datasets/csv/default-ed9c43e8e741b026/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached shuffled indices for dataset at /home/kwamea/.cache/huggingface/datasets/csv/default-04ed86c0369d6106/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-cf14eefb8d952c92.arrow
Loading cached shuffled indices for dataset at /home/kwamea/.cache/huggingface/datasets/csv/default-98e7740e620c6c6f/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-801b7b4a51fce617.arrow
Loading cached shuffled indices for dataset at /home/kwamea/.c

Map:   0%|          | 0/1473 [00:00<?, ? examples/s]

Map:   0%|          | 0/81 [00:00<?, ? examples/s]

Map:   0%|          | 0/81 [00:00<?, ? examples/s]

Map:   0%|          | 0/1473 [00:00<?, ? examples/s]

Map:   0%|          | 0/1473 [00:00<?, ? examples/s]

Map:   0%|          | 0/81 [00:00<?, ? examples/s]

Map:   0%|          | 0/81 [00:00<?, ? examples/s]

Map:   0%|          | 0/81 [00:00<?, ? examples/s]

KeyError: 'text'

## Test the model before finetuning

In [5]:
#Edit eval_prompt to match your data
eval_prompt = """
Summarize this dialog:
A: Hi Tom, are you busy tomorrow’s afternoon?
B: I’m pretty sure I am. What’s up?
A: Can you go with me to the animal shelter?.
B: What do you want to do?
A: I want to get a puppy for my son.
B: That will make him so happy.
A: Yeah, we’ve discussed it many times. I think he’s ready now.
B: That’s good. Raising a dog is a tough issue. Like having a baby ;-) 
A: I'll get him one of those little dogs.
B: One that won't grow up too big;-)
A: And eat too much;-))
B: Do you know which one he would like?
A: Oh, yes, I took him there last Monday. He showed me one that he really liked.
B: I bet you had to drag him away.
A: He wanted to take it home right away ;-).
B: I wonder what he'll name it.
A: He said he’d name it after his dead hamster – Lemmy  - he's  a great Motorhead fan :-)))
---
Summary:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))


Summarize this dialog:
A: Hi Tom, are you busy tomorrow’s afternoon?
B: I’m pretty sure I am. What’s up?
A: Can you go with me to the animal shelter?.
B: What do you want to do?
A: I want to get a puppy for my son.
B: That will make him so happy.
A: Yeah, we’ve discussed it many times. I think he’s ready now.
B: That’s good. Raising a dog is a tough issue. Like having a baby ;-) 
A: I'll get him one of those little dogs.
B: One that won't grow up too big;-)
A: And eat too much;-))
B: Do you know which one he would like?
A: Oh, yes, I took him there last Monday. He showed me one that he really liked.
B: I bet you had to drag him away.
A: He wanted to take it home right away ;-).
B: I wonder what he'll name it.
A: He said he’d name it after his dead hamster – Lemmy  - he's  a great Motorhead fan :-)))
---
Summary:
The speaker is asking the other person to go to the animal shelter with him to get a puppy for his son. They discuss what kind of dog they should get and how they should name 

## Enables Parameter Efficient Finetuning (PEFT)

In [ ]:
#reduces the parameters needed to train
model.train()
def create_peft_config(model):
    from peft import (
        get_peft_model,
        LoraConfig,
        TaskType,
        prepare_model_for_int8_training,
    )

    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=8,
        lora_alpha=32,
        lora_dropout=0.05,
        bias= "none",
        target_modules = ["q_proj", "v_proj"]
    )
    
    kwargs = {
        'use_peft': True, 
        'peft_method': 'lora', 
        'quantization': True, 
        'use_fp16': True, 
        'model_name': os.path.join('/scratch', username, 'models--meta-llama--Llama-2-7b-hf/snapshots/6fdf2e60f86ff2481f2241aaee459f85b5b0bbb9'), 
        'output_dir': os.path.join('/scratch', username)
    }
    
    update_config((train_config, fsdp_config), **kwargs)
    
    model = prepare_model_for_int8_training(model)
    peft_config = generate_peft_config(train_config, kwargs)
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    return model, peft_config

# create peft config
model, lora_config = create_peft_config(model)

/home/kwamea/.conda/envs/llms_env/lib/python3.10/site-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [ ]:
torch.cuda.empty_cache()
from transformers import TrainerCallback
from contextlib import nullcontext
enable_profiler = False
#set up the configurations for training
config = {
    'lora_config': lora_config,
    'learning_rate': 1e-4,
    'num_train_epochs': 1,
    'gradient_accumulation_steps': 2,
    'per_device_train_batch_size': 2,
    'gradient_checkpointing': False,
}

# Set up profiler
if enable_profiler:
    wait, warmup, active, repeat = 1, 1, 2, 1
    total_steps = (wait + warmup + active) * (1 + repeat)
    schedule =  torch.profiler.schedule(wait=wait, warmup=warmup, active=active, repeat=repeat)
    profiler = torch.profiler.profile(
        schedule=schedule,
        on_trace_ready=torch.profiler.tensorboard_trace_handler(f"{output_dir}/logs/tensorboard"),
        record_shapes=True,
        profile_memory=True,
        with_stack=True)
    
    class ProfilerCallback(TrainerCallback):
        def __init__(self, profiler):
            self.profiler = profiler
            
        def on_step_end(self, *args, **kwargs):
            self.profiler.step()

    profiler_callback = ProfilerCallback(profiler)
else:
    profiler = nullcontext()

## Defines training arguments and trains the model

In [ ]:
torch.cuda.empty_cache()
# Define training args
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    bf16=True, 
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    evaluation_strategy="steps",
    logging_steps=5,  # 10
    save_strategy="no",
    optim="adamw_torch_fused",
    auto_find_batch_size = True, 
    max_steps=total_steps if enable_profiler else -1,
    **{k:v for k,v in config.items() if k != 'lora_config'},
    remove_unused_columns=False,
    save_steps=1,
    save_total_limit=5,
    load_best_model_at_end=False
)

with profiler:
    # Create Trainer instance
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=default_data_collator,
        callbacks=[profiler_callback] if enable_profiler else [],
    )
    
# Start training
trainer.train()

## Save the model to output directory

In [ ]:
#saves the model
model.save_pretrained(output_dir)

#Saves model configurations
model.config.to_json_file(os.path.join(output_dir, "config.json"))

#saves PEFT config
peft_config = model.peft_config
json_file_path = os.path.join(output_dir, "peft_config.json")

# Custom serialization function for LoraConfig objects
def lora_config_serializer(obj):
    if isinstance(obj, LoraConfig):
        # Return a dictionary representation of the LoraConfig object
        return obj.__dict__
    raise TypeError("Type not serializable")

# Write the dictionary to the JSON file using the custom serializer
with open(json_file_path, "w") as json_file:
    json.dump(peft_config, json_file, default=lora_config_serializer, indent=4)

## Test model on the same input as before

In [ ]:
#Tests the model on the sample input from before
model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))


## Load Saved Model

In [18]:
#load and test- get metrics and automatically show random 5 model input and output
#get this cell running on cold
model = PeftModelForCausalLM.from_pretrained(model, output_dir)
#import nltk
#from nltk.translate.bleu_score import corpus_bleu

# Assuming references is a list of reference texts and hypotheses is a list of generated texts
'''
def compute_bleu(references, hypotheses):
    # Compute BLEU score using nltk's corpus_bleu function
    bleu_score = corpus_bleu(references, hypotheses)
    return bleu_score
'''

# Define data collator for evaluation
eval_data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Create Trainer instance for evaluation
eval_training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_eval_batch_size=8,  # Adjust batch size based on your system's memory
    remove_unused_columns=False,
    load_best_model_at_end=False,
)
eval_trainer = Trainer(
    model=model,
    args=eval_training_args,
    data_collator=eval_data_collator,
)

# Evaluate the model on test_dataset
evaluation_results = eval_trainer.predict(test_dataset)
'''
eval_prompt = """
Summarize this dialog:
A: Hi Tom, are you busy tomorrow’s afternoon?
B: I’m pretty sure I am. What’s up?
A: Can you go with me to the animal shelter?.
B: What do you want to do?
A: I want to get a puppy for my son.
B: That will make him so happy.
A: Yeah, we’ve discussed it many times. I think he’s ready now.
B: That’s good. Raising a dog is a tough issue. Like having a baby ;-) 
A: I'll get him one of those little dogs.
B: One that won't grow up too big;-)
A: And eat too much;-))
B: Do you know which one he would like?
A: Oh, yes, I took him there last Monday. He showed me one that he really liked.
B: I bet you had to drag him away.
A: He wanted to take it home right away ;-).
B: I wonder what he'll name it.
A: He said he’d name it after his dead hamster – Lemmy  - he's  a great Motorhead fan :-)))
---
Summary:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))
'''

'\neval_prompt = """\nSummarize this dialog:\nA: Hi Tom, are you busy tomorrow’s afternoon?\nB: I’m pretty sure I am. What’s up?\nA: Can you go with me to the animal shelter?.\nB: What do you want to do?\nA: I want to get a puppy for my son.\nB: That will make him so happy.\nA: Yeah, we’ve discussed it many times. I think he’s ready now.\nB: That’s good. Raising a dog is a tough issue. Like having a baby ;-) \nA: I\'ll get him one of those little dogs.\nB: One that won\'t grow up too big;-)\nA: And eat too much;-))\nB: Do you know which one he would like?\nA: Oh, yes, I took him there last Monday. He showed me one that he really liked.\nB: I bet you had to drag him away.\nA: He wanted to take it home right away ;-).\nB: I wonder what he\'ll name it.\nA: He said he’d name it after his dead hamster – Lemmy  - he\'s  a great Motorhead fan :-)))\n---\nSummary:\n"""\n\nmodel_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")\n\nmodel.eval()\nwith torch.no_grad():\n    print(toke

In [34]:
print(testSet[0])

{'text': "Summarize this dialog:\nClaire: <file_photo>\r\nKim: Looks delicious...\r\nLinda: No way... Look what I'm cooking right now:\r\nLinda: <file_photo>\r\nClaire: hahahaha \r\nKim: Curry dream team\r\nClaire: Enjoy your dinner :*\n---\nSummary:\nBoth Claire and Linda are making curry for dinner. </s>"}


In [33]:
for _ in range(5):
    random_idx = random.randint(0, len(test_dataset) - 1)
    eval_prompt = testSet[random_idx]['text']
    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
    print("Input: \n", eval_prompt)
    model.eval()
    print("Generated Output: \n")
    with torch.no_grad():
        print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

Input: 
 Summarize this dialog:
Malik: have you heard of that paleo diet?
Malik: i need to lose some weight and i really want to try it
Samantha: i've heard of it but i've also heard about the keto diet
Samantha: AAAAANNNDDDD... i also need to lose weight lol
Malik: what are you talking about?!? lol
Malik: you're so skinny
Samantha: whatever :-)
Malik: should we try one of those together?
Malik: it's always easier when someone's doing it with you
Samantha: YES!!!!
Malik: we can also go for runs together like we used to :-D
Samantha: let's do it!! i'm so pumped!
Malik: so paleo or keto?
Samantha: what's the difference?
Malik: i think they're practically the same, but you can't have dairy on paleo
Samantha: can you have dairy on keto?
Malik: i think you can, i'm no sure though
Samantha: ok let me go online and read more about this
Samantha: and i'll text you back later with more info
Malik: ok
Malik: are you excited??
Samantha: i really am!!!!!!!!! :-D
---
Summary:
Malik and Samanta want